# Парсинг МАС
#### Анастасия Костяницына, Александра Дахина 
28 декабря 2018

In [18]:
from bs4 import BeautifulSoup
from bs4 import Tag
import re
from string import punctuation, digits
from collections import defaultdict
import os
import pandas as pd
import numpy as np
from copy import deepcopy
import copy

In [2]:
from string import punctuation, digits
punctuation = set(punctuation + '«»—…“”\n\t' + digits)

Пример структуры для грам. значений:

<superEntry>
    <metalemma></metalemma>
    <entry>
        <form>
            + <orth></orth>
            + <usg type="time"> temporal, historical era (‘archaic’, ‘old’, etc.) </usg>
            + <usg type="reg"> register = книжн, разг, </usg>
            + <usg type="style"> style (figurative, literal, etc.) + ирон., негативн., положит. </usg>
            + <usg type="dom"> domain, сфера деятельности</usg>
        </form>
        <gramGrp>
            + <pos> ч.речи </pos>
            + <gender>род</gender>
            + <num>число</num>"
            +<per>лицо</per>
            + <mood>наклонение</mood>
            + <asp> вид (совершенный/несовершенный) </asp>
            + <transit> переходный/непереходный </transit>
            <refl> возвратность </refl>
            + <pron_type> личное/неличное </pron_type>
        </gramGrp>
        <form type="inflected">
            + <case> ...</case>
            + <num>число</num>"
            <tns>...</tns>
            +  <orth  extent="part"> здесь изменяемая часть, например, если в словаре написано "мать, -ери", то в этот тэг пишем -ери и указываем extent="part", если в словаре написано "мать, матери", то сюда пишем "матери" и extent не указываем  </orth>
            + <orth  > если указано несколько форм, можно указать несколько orth, например для словарного входа "take; took, taken" можно написать сюда took и taken</orth>
        </form>
    </entry>
</superEntry>

## Функции:

In [3]:
def add_lemma(cur_soup, unique_entry):
    
    '''
    Добавляет лемму в шаблон
    '''

    cur_soup.metalemma.string = unique_entry
    cur_soup.orth.string = unique_entry
    cur_soup.orth['main'] = "True"

In [4]:
def add_pos(entr_soup, entry, data):
    
    '''
    Добавляет часть речи в шаблон
    '''

    ps = data.loc[data['mas'] == entry[0]]['_PoS'].tolist()[0]
    if ps != []:
        entr_soup.gramgrp.append(cur_soup.new_tag('pos'))
        entr_soup.gramgrp.pos.string = ps

In [5]:
def new_tag(parent, child, entr_soup, val, name='type', attr=False):
    
    '''
    Создает новый тег по заданным пар-ам
    '''
    
    new = entr_soup.new_tag(child)
    if attr:
        new[name] = attr
    new.string = val
    parent.append(new)

In [6]:
def searching(entr_soup, data, item, name, index=False):
    
    '''
    Ищет в столбце информацию, которая заключена в тегах b, u, i
    Ищет окончания слов (например, -а)
    
    b - вариант написания
    i - род, число, стиль, часть речи, вид, падеж
    u - формы слова 
    
    parent - то, куда потом нужно будет вставлять информацию, найденную в тегах
    '''
    
    gr = data.loc[data['mas'] == item][name].tolist()[0]
    if gr == '':
        return None, None

    if index is False:
        parent_f = entr_soup.find("form")
        parent_fi = entr_soup.find("form", type="inflected")
        parent_gr = entr_soup.find("gramgrp")
        parent = (parent_f, parent_fi, parent_gr)
        adding_soup = entr_soup.entry
    
    else:
        parent_sns = entr_soup.find("sense", n=str(index+1))
        parent_gr = parent_sns.find("gramgrp")
        parent = (parent_sns, parent_gr)
        adding_soup = parent_sns
        
    b = re.findall('\[b\](.*?)\[/b\]', gr)
    i = re.findall('\[i\](.*?)\[/i\]', gr)
    u = re.findall('\[u\](.*?)\[/u\]', gr)
    non = re.findall('(\-[a-zA-Zа-яА-Я]+?)\\b', gr)
    
    together = [b, i, u, non]
    
    if together == [[], [], [], []]:
        new_tag(adding_soup, 'note', entr_soup, gr)
    
    return parent, together

In [7]:
def in_tags(log, entr_soup, data, entry, tags, parent_fi, parent_f, parent_gr, adding_soup, index=False):
    
    '''
    Вставляет найденную информацию в тегах таблицы в нужное место шаблона
    '''

    global punctuation, dom_values, gender_values, num_values, case_values, reg_values, time_values, per_values, style_values, asp_values, transit_values, mood_values, pron_type_values 
  
    b, i, u, non = tags
    
    for val in non:
        if val not in log['orth_part']:
            new_tag(parent_fi, 'orth', entr_soup, val, name='extent', attr='part')
            log['orth_part'].add(val)
        
    
    for val in b: 
        val = val.strip(''.join(punctuation))
        if val not in log['orth']:
            new_tag(parent_f, 'orth', entr_soup, val)
            log['orth'].add(val)
    
    for val in u:
        if len(val) >= 2:
            if val[:2] == data.loc[data['mas'] == entry[0]]['Lemma.1'].tolist()[0][:2]:
                if val not in log['orth']:
                    new_tag(parent_fi, 'orth', entr_soup, val)
                    log['orth'].add(val)
            else:
                if '-' not in val:
                    val = '-' + val
                if val not in log['orth_part']:
                    new_tag(parent_fi, 'orth', entr_soup, val, name='extent', attr='part')
                    log['orth_part'].add(val)
    
    for val in i:

        val = val.lower()
        gender = re.findall(gender_values, val)
        num = re.findall(num_values, val)
        reg = re.findall(reg_values, val)
        time = re.findall(time_values, val)
        style = re.findall(style_values, val)
        dom = re.findall(dom_values, val)
        asp = re.findall(asp_values, val)
        transit = re.findall(transit_values, val)
        mood = re.findall(mood_values, val)
        pron_type = re.findall(pron_type_values, val)
        per = re.findall(per_values, val)
        
        for g in gender: new_tag(parent_gr, 'gender', entr_soup, g)
        for g in num: new_tag(parent_gr, 'num', entr_soup, g)
        for g in reg: new_tag(parent_f, 'usg', entr_soup, g, attr="reg")
        for g in time: new_tag(parent_f, 'usg', entr_soup, g, attr="time")
        for g in style: new_tag(parent_f, 'usg', entr_soup, g, attr="style")
        for g in dom: new_tag(parent_f, 'usg', entr_soup, g, attr="dom")
        for g in asp: new_tag(parent_gr, 'asp', entr_soup, g)
        for g in transit: new_tag(parent_gr, 'transit', entr_soup, g)
        for g in mood: new_tag(parent_gr, 'mood', entr_soup, g)
        for g in pron_type: new_tag(parent_gr, 'pron_type', entr_soup, g)
        for g in per: new_tag(parent_gr, 'per', entr_soup, g)
            
        together = [gender, num, reg, time, style, dom, asp, transit, mood, pron_type, per]
            
        if index is False:
            case = re.findall(case_values, val)
            for g in case: new_tag(parent_fi, 'case', entr_soup, g)
            together.append(case)

        if together == [[] for _ in range(len(together))]:
            new_tag(adding_soup, 'note', entr_soup, val)
        

In [8]:
def add_gram(entr_soup, entry, data, name):
    '''
    Добавляет информацию из столбика (GrAll)
    '''

    log = defaultdict(set)
    parent, tags = searching(entr_soup, data, entry[0], name)
    
    if (parent, tags) != (None, None):
        parent_f, parent_fi, parent_gr = parent
        in_tags(log, entr_soup, data, entry, tags, parent_fi, parent_f, parent_gr, entr_soup, index=False)
    

In [9]:
def add_gram2_3(entr_soup, entry, data, name):
    
    '''
    Добавляет информацию из столбика (Gr2) или (Gr3)
    '''
    
    for index, item in enumerate(entry):

        log = defaultdict(set)
        parent, tags = searching(entr_soup, data, item, name, index=index)
        if (parent, tags) != (None, None):
            parent_sns, parent_gr = parent
            in_tags(log, entr_soup, data, entry, tags, parent_sns, parent_sns, parent_gr, parent_sns, index=index)

In [15]:
def create_entry():
    
    '''
    Если слово может иметь несколько частей речи, они оформляются как hom.
    Функуия сосздает дополнительные entry
    '''
    
    ex = '<entry><form><orth></orth></form><gramGrp></gramGrp><form type="inflected"></form><sense></sense></entry>'
    soup = BeautifulSoup(ex, 'lxml')
    soup.entry['type'] = 'hom'
    return soup

Регулярные выражения для поиски информации

In [12]:
dom_values = '\\b(Сад\.|с\.\-х|Спец\.|Архит\.|Авиа\.|Опт\.|астр\.|Мед\.|Зоол\.|этногр\.|ист\.|Физ\.|хим\.|Мат\.|Обл\.|Фин\.|Ж\.-д\.|Астр\.|Психол\.|Грамм\.|лит\.|Полигр\.|Тех\.|Дипл\.|Муз\.|Биол\.|Пед\.|Филос\.|Метеор\.|Бухг\.|Церк\.|Трад\.\-поэт\.|Лингв\.|Юр\.|Геол\.|Биохим\.|Анат\.|Охот\.|Кулин\.|Воен\.)'.lower()
gender_values = '\\b(ж\.|м\.|ср\.)'
num_values = '\\b(мн\.|ед\.)'
case_values = '\\b(им\.|род\.|вин\.|винительным|дат\.|предл\.|твор\.|творительным)'
reg_values = '\\b(Разг\.|Книжн\.|Прост\.|прост\.|книжн\.|разг\.|Груб\. прост\.|Прост\. пренебр\.)'.lower()
time_values = '\\b(устар\.)'
style_values = '\\b(ирон\.|высок\.)'
asp_values = '\\b(сов\.|несов\.)'
transit_values = '\\b(перех\.|неперех\.)'
mood_values = '\\b(повел\.)'
pron_type_values = '\\b(личн\.|неличн\.)'
per_values = '\\b([1-3] ?л\.)'

Шаблон вхождения

In [14]:
xml_n = '<superEntry><metalemma></metalemma><entry><form><orth></orth></form><gramGrp></gramGrp><form type="inflected"></form><sense></sense></entry></superEntry>'


## Данные

In [16]:
data = pd.read_csv('MAS/test.csv',  sep=';', na_values= np.nan)
#data = pd.read_csv('/Users/Stoneberry/Desktop/Uni/Мастерская/MAS/masR_Ja.csv',  sep=';', na_values= np.nan)
data = data.fillna('')

In [172]:
data.head()

,1,mas,Lex,Entr,Lemma,(Gr),(GrAll),(n),(Gr2),Lemma.1,_PoS,n,МУ,(Def),(Gr3),(Ex),Unnamed: 16
0,371,масР0198A,раз...,РАЗ…,раз,"([i]а также[/i] [b]разо...[/b], [b]разъ...[/b]...","([i]а также[/i] [b]разо...[/b], [b]разъ...[/b]...",%n=IА,,раз,COM,IА,,Употребляется при образовании глаголов и обозн...,,,
1,372,масР0198B,,,раз,,"([i]а также[/i] [b]разо...[/b], [b]разъ...[/b]...",%n=IБ,,раз,COM,IБ,,Образует форму совершенного вида некоторых гла...,,,
2,373,масР0198C,,,раз,,"([i]а также[/i] [b]разо...[/b], [b]разъ...[/b]...",%n=II,,раз,COM,II,,Употребляется при образовании прилагательных и...,,,
3,374,масР0198D,,,раз,,"([i]а также[/i] [b]разо...[/b], [b]разъ...[/b]...",%n=III,,раз,COM,III,,Употребляется при образовании существительных ...,,,
4,375,масР0199A,раз1,РАЗ1,раз1,"-а (-у), [i]мн.[/i] [u]разы'[/u], [u]раз[/u], ...","-а (-у), [i]мн.[/i] [u]разы'[/u], [u]раз[/u], ...",%n=1,,раз,С,1,,В сочетании с числительным «один» или без него...,,"[i]— Жизнь дается один раз, и хочется прожить ...",


Чтобы обеспечить простоту парсинга и сохранность структуры, групируем строки таблицы по леммам так, чтобы для каждой леммы отдельно были сгруппированы строки по части речи.

In [19]:
groups = data.groupby('Lemma.1')['mas'].apply(list)
d = defaultdict(list)

# каждое слово может иметь несколько омографов, группируем по ним
for unique_entry in groups:
    lemma = data.loc[data['mas'] == unique_entry[0]]['Lemma.1'].tolist()[0]
    new = data.loc[data['mas'].isin(unique_entry)]
    new_groups = new.groupby('Lemma')['mas'].apply(list)
    d[lemma] = [i for i in new_groups]

In [20]:
data.loc[data['Lemma.1'] == 'раз']['_PoS']

0            COM
1            COM
2            COM
3            COM
4              С
5              С
6              С
7              С
8              С
9              С
10    ПРЕДИКАТИВ
11         НАРЕЧ
12          СОЮЗ
Name: _PoS, dtype: object

In [21]:
d['раз']

[['масР0198A', 'масР0198B', 'масР0198C', 'масР0198D'],
 ['масР0199A',
  'масР0199B',
  'масР0199C',
  'масР0199D',
  'масР0199E',
  'масР0199F'],
 ['масР0200A'],
 ['масР0201A'],
 ['масР0202A']]

Все распаршенные вхождения

In [22]:
all_ = [] 

Проходимся по таблице

In [23]:
for unique_entry in d:

    cur_soup = BeautifulSoup(copy.copy(xml_n), 'lxml')
    add_lemma(cur_soup, unique_entry)
    
    for index, entry in enumerate(d[unique_entry]):

        not_main = False

        if index == 0:
            cur_soup.entry['type'] = 'main' 
            entr_soup = cur_soup 
        else: 
            entr_soup = create_entry() # entr_soup - entry каждого значения
            entr_soup.orth.string = unique_entry

        add_pos(entr_soup, entry, data)
        add_gram(entr_soup, entry, data, '(GrAll)')
        add_gram2_3(entr_soup, entry, data, '(Gr2)')
        add_gram2_3(entr_soup, entry, data, '(Gr3)')
        cur_soup.superentry.append(entr_soup.entry)

    all_.append(cur_soup)